<a href="https://colab.research.google.com/github/audreychristensen/Bird_Audio_CNN/blob/main/1.2%20Process%20All%20Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import h5py
import traceback
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import librosa
import librosa.display
import IPython.display as ipd
from PIL import Image
import soundfile as sf
import scipy.io.wavfile as wave
import scipy.ndimage as ndimage
import scipy.stats as stats
from scipy import interpolate

In [ ]:
base_dir = '/content/drive/MyDrive/F2024/Applied Data Science/Project 3/'
src_dir = '/content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/'
hdf5_path = base_dir + "output_spectrograms_final.h5"

# Preparing Bird Files

In [ ]:
# get all bird species directories

birds_list = [src_dir + bird + '/' for bird in sorted(os.listdir(src_dir))]

In [ ]:
birds_over_50_files = []
for bird in birds_list:
    # Get the list of files in the bird's directory (excluding subdirectories)
    files = [f for f in os.listdir(bird) if os.path.isfile(os.path.join(bird, f))]

    # Check if the number of files exceeds 50
    if len(files) > 50:
        birds_over_50_files.append(bird)

In [ ]:
bird_names = [os.path.basename(os.path.normpath(bird)) for bird in birds_over_50_files]
bird_dict = {bird_name: (idx + 1) for idx, bird_name in enumerate(set(bird_names))}
bird_dict['noise'] = 0

In [ ]:
birds = birds_over_50_files

In [ ]:
pd.Series(bird_dict).to_csv(base_dir + 'bird_dict.csv')

# Functions

In [ ]:

def getMelSpec(path, seconds = 5, overlap = 4, minlen = 3, winlen=0.05, winstep=0.0097, NFFT=840, sr_target=44100):
  """
  Takes in a filepath (mp3) and outputs mel spectrograms
  """
  y, sr = librosa.load(path, sr=sr_target)
  print(f"shape: {y.shape}", f'sr: {sr}')
  sig_splits = []
  step = int((seconds - overlap) * sr)
  window_length = int(seconds * sr)
  NFFT = max(NFFT, int(winlen * sr))

  for i in range(0, len(y), step):
    split =  y[i:i + window_length]
    if len(split) >= minlen:
      sig_splits.append(split)

  if len(sig_splits) == 0:
    sig_splits.append(sig)


  for split_sig in sig_splits:
        # compute mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=split_sig,
            sr=sr,
            n_fft=int(winlen * sr),
            hop_length=int(winstep * sr),
            n_mels=128*2
        )

        # convert to dB scale
        mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

        # resize to fixed shape
        mel_spec_resized = cv2.resize(mel_spec_db, (512, 256))

        yield mel_spec_resized


In [ ]:
def filter_isolated_cells(array, struct):
    """
    Borrowed from BirdCLEF Paper
    """
    filtered_array = np.copy(array)
    id_regions, num_ids = ndimage.label(filtered_array, structure=struct)
    id_sizes = np.array(ndimage.sum(array, id_regions, range(num_ids + 1)))
    area_mask = (id_sizes == 1)
    filtered_array[area_mask[id_regions]] = 0

    return filtered_array

In [ ]:
def hasBird(spec, threshold=16):
    """
    Borrowed from BirdCLEF Paper
    """
    #working copy
    img = spec.copy()

    #STEP 1: Median blur
    img = cv2.medianBlur(img,5)

    #STEP 2: Median threshold
    col_median = np.median(img, axis=0, keepdims=True)
    row_median = np.median(img, axis=1, keepdims=True)

    img[img < row_median * 3] = 0
    img[img < col_median * 4] = 0
    img[img > 0] = 1

    #STEP 3: Remove singles
    img = filter_isolated_cells(img, struct=np.ones((3,3)))

    #STEP 4: Morph Closing
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((5,5), np.float32))

    #STEP 5: Frequency crop
    img = img[128:-16, :]

    #STEP 6: Count columns and rows with signal
    #(Note: We only use rows with signal as threshold, but columns might come in handy in other scenarios)

    #column has signal?
    col_max = np.max(img, axis=0)
    col_max = ndimage.morphology.binary_dilation(col_max, iterations=2).astype(col_max.dtype)
    cthresh = col_max.sum()

    #row has signal?
    row_max = np.max(img, axis=1)
    row_max = ndimage.morphology.binary_dilation(row_max, iterations=2).astype(row_max.dtype)
    rthresh = row_max.sum()

    #final threshold
    thresh = rthresh

    #DBUGB: show?
    #print thresh
    #cv2.imshow('BIRD?', img)
    #cv2.waitKey(-1)

    #STEP 7: Apply threshold (Default = 16)
    bird = True
    if thresh < threshold:
        bird = False

    return bird, thresh

In [ ]:
def exceeds_max_duration(wav_file, max_duration):
    # Load the WAV file
    y, sr = librosa.load(wav_file, sr=None)  # sr=None keeps the original sampling rate
    duration = librosa.get_duration(y=y, sr=sr)
    return duration > max_duration

In [ ]:
def create_new_hdf5(file_count):
    hdf5_path = os.path.join(base_dir, f"birds_data_{file_count}.h5")
    return h5py.File(hdf5_path, "w")

# Get spectrograms and save as h5py file

In [ ]:
#specify maximum number of spectrograms per species (-1 = No limit)
MAX_SPECS = 500

#limit number of species? (None = No limit)
MAX_SPECIES = None

# define the maximum allowed duration in seconds
MAX_DURATION = 60

In [ ]:
# start counters for h5py file creation

bird_count = 0
file_count = 1

In [ ]:
h5file = create_new_hdf5(file_count)
noise_group = h5file.create_group("noise")

for bird in birds:
    bird_name = os.path.basename(os.path.normpath(bird))

    # check if we need a new HDF5 file (every 20 birds)
    if bird_count >= 20:
        h5file.close()
        file_count += 1
        bird_count = 0  # reset bird counter

        # Create a new HDF5 file
        h5file = create_new_hdf5(file_count)
        noise_group = h5file.create_group("noise")
        print(f"Created new HDF5 file: birds_data_{file_count}.h5")

    # create a group for the bird
    bird_group = h5file.create_group(bird_name)
    total_specs = 0

    # get all WAV files
    wav_files = [os.path.join(bird, wav) for wav in sorted(os.listdir(bird))]

    # parse WAV files
    for wav in wav_files:
        if exceeds_max_duration(wav, MAX_DURATION):
            print(f"Skipping {wav}, as it exceeds the maximum duration of {MAX_DURATION} seconds.")
            continue  # skip this file and move to the next

        spec_cnt = 0

        try:
            # get every spectrogram from each WAV file
            for spec in getMelSpec(wav):
                # does spec contain bird sounds?
                isbird, thresh = hasBird(spec)

                # choose the group for the spectrogram
                group = bird_group if isbird else noise_group

                # save the spectrogram as a dataset in HDF5
                dataset_name = f"{os.path.basename(wav).rsplit('.')[0]}_{spec_cnt}_{thresh}"
                group.create_dataset(
                    dataset_name,
                    data=spec,
                    compression="gzip",
                    compression_opts=9,
                )

                spec_cnt += 1
                total_specs += 1

            # exceeded spec limit?
            if total_specs >= MAX_SPECS and MAX_SPECS > -1:
                print("Exceeded max specs.")
                break

            print(f"SPECS: {spec_cnt}")

        except Exception as e:
            print(f"Error processing {wav}: {e}")
            traceback.print_exc()
            pass

    bird_count += 1

# close the final HDF5 file
h5file.close()
print("HDF5 files created successfully.")


shape: (639360,) sr: 44100


<ipython-input-10-8628287be95a>:33: DeprecationWarning: Please import `binary_dilation` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  col_max = ndimage.morphology.binary_dilation(col_max, iterations=2).astype(col_max.dtype)
<ipython-input-10-8628287be95a>:38: DeprecationWarning: Please import `binary_dilation` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  row_max = ndimage.morphology.binary_dilation(row_max, iterations=2).astype(row_max.dtype)


SPECS: 15
shape: (329341,) sr: 44100
SPECS: 8
shape: (354125,) sr: 44100


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2205 is too large for input signal of length=1325
  warnings.warn(


SPECS: 9
shape: (2023951,) sr: 44100
SPECS: 46
shape: (772224,) sr: 44100
SPECS: 18
shape: (609639,) sr: 44100
SPECS: 14
shape: (2318976,) sr: 44100
SPECS: 53
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/AmericanCrow/568327.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/AmericanCrow/568365.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (1428023,) sr: 44100
SPECS: 33
shape: (1841732,) sr: 44100
SPECS: 42
shape: (241575,) sr: 44100
SPECS: 6
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/AmericanCrow/574517.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (1978258,) sr: 44100
SPECS: 45
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/AmericanCrow/588916.mp3, as it exceeds the maximum duration of 60 seconds.
shape: 

...

shape: (384285,) sr: 44100
SPECS: 9
shape: (1975565,) sr: 44100
SPECS: 45
shape: (2574792,) sr: 44100
SPECS: 59
shape: (214259,) sr: 44100
SPECS: 5
shape: (173038,) sr: 44100
SPECS: 4
shape: (2380687,) sr: 44100
SPECS: 54
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/White-throatedSparrow/729148.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/White-throatedSparrow/729186.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (666321,) sr: 44100
SPECS: 16
shape: (282012,) sr: 44100
SPECS: 7
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/White-throatedSparrow/730544.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (1819089,) sr: 44100
Exceeded max specs.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/554196.mp3

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2205 is too large for input signal of length=511
  warnings.warn(


shape: (2093163,) sr: 44100
SPECS: 48
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/658712.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (1831121,) sr: 44100
SPECS: 42
shape: (1990397,) sr: 44100
SPECS: 46
shape: (515066,) sr: 44100
SPECS: 12
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/692206.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/692207.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/692208.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/WoodThrush/692210.mp3, as it exceeds the maximum duration of 60 seconds.
shap

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2205 is too large for input signal of length=421
  warnings.warn(


Exceeded max specs.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/Yellow-breastedChat/549193.mp3, as it exceeds the maximum duration of 60 seconds.
shape: (2027895,) sr: 44100
SPECS: 46
shape: (814968,) sr: 44100
SPECS: 19
shape: (200310,) sr: 44100
SPECS: 5
shape: (1554790,) sr: 44100
SPECS: 36
shape: (1034897,) sr: 44100
SPECS: 24
shape: (864596,) sr: 44100
SPECS: 20
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/Yellow-breastedChat/559240.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/Yellow-breastedChat/563101.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/Yellow-breastedChat/566182.mp3, as it exceeds the maximum duration of 60 seconds.
Skipping /content/drive/MyDrive/F2024/Applie